In [ ]:
%matplotlib notebook
import os
import re
import sys
from glob import glob

os.environ['CUDA_VISIBLE_DEVICES'] = '0
import keras
import keras.layers as layers
import matplotlib.colors
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tqdm
from keras.models import Model
from keras.utils import to_categorical
from keras_applications.mobilenet_v2 import MobileNetV2  # , preprocess_input
from skimage.io import imread, imsave
from skimage.transform import resize
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
def get_mask_name(filename):
    groups = re.match("(\d)+_IMG_(\d+).tiff", filename).groups()
    return "mask1_IMG_{:s}.tiff".format(groups[1])

def resize(image, output_shape):
    """Fast resize using PIL (ensure pillow-simd is installed instead of pillow)"""
    image = PIL.Image.fromarray(image)
    image = image.resize(output_shape[:2])
    return np.array(image)


In [ ]:
IMAGE_DIR = "/data/standard/ColorCheckerRECommended/5D - part 1"
MASK_DIR = "/data/standard/ColorCheckerRECommended/5D - part 1/masks"
INPUT_SHAPE = (224, 224, 3)
RESIZE_SHAPE = INPUT_SHAPE[:2]

images, masks, categories = [], [], []
image_paths = glob(IMAGE_DIR +  "/*.tiff")
print("Loading {:d} positive images".format(len(image_paths)))
for image_path in tqdm.tqdm_notebook(image_paths):
    filename = os.path.basename(image_path)
    mask_name = get_mask_name(filename)
    images.append(resize(imread(os.path.join(IMAGE_DIR, filename)), output_shape=RESIZE_SHAPE))
    masks.append(resize(imread(os.path.join(MASK_DIR, mask_name))[..., 0], output_shape=RESIZE_SHAPE) < 0.5)
categories += [1] * len(image_paths)

In [ ]:
# Load application specific negatives
image_paths = glob("/data/acfr/ladybird/sym/all/images/cc_det_train/*.jpg")
for image_path in tqdm.tqdm_notebook(image_paths):
    filename = os.path.basename(image_path)
    image = resize(imread(image_path), output_shape=RESIZE_SHAPE)
    images.append(image)
    masks.append(np.zeros(image.shape[:2], dtype=np.uint8))
categories += [0] * len(image_paths)

# Load application specific positives
image_paths = glob("/data/acfr/ladybird/sym/all/images/cc_det_train/positive/*.jpg")
for image_path in tqdm.tqdm_notebook(image_paths):
    filename = os.path.basename(image_path)
    image = resize(imread(image_path), output_shape=RESIZE_SHAPE)
    images.append(image)
    masks.append(np.zeros(image.shape[:2], dtype=np.uint8))
categories += [1] * len(image_paths)

In [ ]:
print(len(images), len(masks), len(categories))
print("images", images[0].shape, images[-1].shape)
print("masks", masks[0].shape, masks[-1].shape)
assert images[0].shape == images[-1].shape
assert masks[0].shape == masks[-1].shape
assert len(images) == len(masks) == len(categories)
images = np.array(images, dtype=np.uint8)
masks = np.array(masks, dtype=np.uint8)[..., np.newaxis]
categories = np.array(categories, dtype=np.uint8)
categories = to_categorical(categories, num_classes=2)
print(images.shape, masks.shape, categories.shape)

In [ ]:
plt.figure()
plt.subplot(1, 2, 1)
plt.hist(images[:482].reshape([-1, 3]), log=True)
plt.subplot(1, 2, 2)
plt.hist(images[482:].reshape([-1, 3]), log=True)

In [ ]:
# Load application specific negatives
def get_random_blind():
    test_blind_dir = glob("/data/acfr/ladybird/sym/all/images/rgb_small_jpg/left/*.jpg")
    image_path = np.random.choice(test_blind_dir)
    filename = os.path.basename(image_path)
    return resize(imread(image_path), output_shape=RESIZE_SHAPE)


In [ ]:
from imgaug import augmenters as iaa
from itertools import zip_longest

def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

seq_geom = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.Flipud(0.5), # horizontally flip 50% of the images
    iaa.Affine(
        scale={"x": (0.9, 1.2), "y": (0.9, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    ),
])
seq_col = iaa.Sequential([
    iaa.ContrastNormalization((0.9, 1.1)),
    iaa.Multiply((0.9, 1.1), per_channel=0.2),
    iaa.Add((-10, 10), per_channel=0.5),
#     iaa.Grayscale(alpha=(0.5, 1.0)),
])

def aug_gen(x, y, batch_size, fillvalue=None, augment=False):
    while True:
        for images, masks in zip(grouper(x, batch_size, fillvalue), grouper(y, batch_size, fillvalue)):
            xx = np.array([xxx for xxx in images if xxx is not None])
            yy = np.array([yyy for yyy in masks if yyy is not None])
            
            if augment:
                seq_det = seq_geom.to_deterministic()
                xx = seq_det.augment_images(xx)
                xx = seq_col.augment_images(xx)
            yield xx, yy
            
def cce_from_sigmoid_seg_gen(gen):
    for images, masks in gen:
        masks_cce = np.concatenate([masks <= 0, masks > 0], axis=-1)
        yield images, masks_cce
        
def classification_from_seg(gen, output='sigmoid'):
    for images, masks in gen:
        if output == 'sigmoid':
            classes = np.array([
                1 if np.count_nonzero(batch) else 0
                for batch in masks])
        elif output == 'softmax':
            classes = np.array(
                [[0, 1] if np.count_nonzero(batch) else [1, 0]
                for batch in masks])
        yield images, classes
        
# def preprocess_gen(gen):
#     for images, masks in gen:
#         yield preprocess_input(images.astype(np.float32)), masks
        


In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(images, categories, shuffle=True, test_size=0.2)
[i.setflags(write=False) for i in [X_train, X_test, y_train, y_test]]
X_train.shape, y_train.shape, X_test.shape, y_test.shape
print("train", np.count_nonzero(y_train.argmax(-1) == 1) / y_train.shape[0])
print("test", np.count_nonzero(y_test.argmax(-1) == 1) / y_test.shape[0])

found = [0, 0]
plt.figure()
for xx, yy in aug_gen(X_train, y_train, batch_size=1, augment=True):
    class_id = int(yy.argmax(-1).ravel())
    if not found[class_id]:
        print(xx.shape, yy.shape, xx.dtype, yy.dtype)
        print(xx.min(), xx.max(), yy.min(), yy.max())
        print(xx[0].min(), xx[0].mean(), xx[0].max(), xx[0].std())
        image = np.clip(100*np.log10(1+xx[0]), 0, 255).astype(np.uint8)
        plt.subplot(1, 2, 1 + class_id)
        plt.imshow(image)
        plt.title(str(class_id))
        found[class_id] += 1
    if all(found):
        break

In [ ]:
from keras.models import Sequential

In [ ]:
def model_preprocess(model):
    seq = Sequential([
        layers.InputLayer(input_shape=INPUT_SHAPE, dtype='float32'),
        layers.Lambda(lambda x: x / 127.5 - 1, output_shape=INPUT_SHAPE),
        model
    ])
    return seq

def make_model(weights, heads_only=False):
    x_in = layers.Input(shape=INPUT_SHAPE, dtype='float32')
    x = x_in
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), activation='relu', dilation_rate=1)(x)
    x = layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu', dilation_rate=6)(x)
    x = layers.Conv2D(128, (3, 3), strides=(1, 1), activation='relu', dilation_rate=24)(x)
    x = layers.GlobalAveragePooling2D()(x)
    y = layers.Dense(2, activation='softmax')(x)
    model = Model(x_in, y)
    return model


In [ ]:
def make_model(weights, heads_only=True):
    global model
    del model
    keras.backend.clear_session()
    model = MobileNetV2(input_shape=INPUT_SHAPE, include_top=False, weights=weights)
    x = model.output
#     x = model.get_layer("block_12_add").output
    x = layers.GlobalAveragePooling2D()(x)
    y = layers.Dense(2, activation='softmax')(x)
    model = Model(model.input, y)
    if heads_only:
        for layer in model.layers[:-3]:
            layer.trainable = False
    return model

In [ ]:
# def make_model(weights, heads_only=True):
#     global model
#     del model
#     keras.backend.clear_session()
#     model = MobileNetV2(input_shape=INPUT_SHAPE, include_top=False, weights=weights)
#     x = model.output
# #     x = model.get_layer("block_12_add").output
#     x = layers.GlobalAveragePooling2D()(x)
#     x = layers.Dense(2)(x)
#     y = layers.Activation('sigmoid')(x)
#     model = Model(model.input, y)
#     if heads_only:
#         trainable_layers = model.layers[-3:] + [layer for layer in model.layers[:-3] if False]
#         untrainable_layers = [layer for layer in model.layers if layer not in trainable_layers]
#         for layer in untrainable_layers:
#             layer.trainable = False
#         print("Trainable layers:")
#         print([layer.name for layer in trainable_layers])
#         print("Untrainable layers:")
#         print([layer.name for layer in untrainable_layers])
#     model.compile('nadam', 'binary_crossentropy',
#                   metrics=['acc', auc_factory('PR')])
#     return model


In [ ]:
model = None

model = make_model(weights='imagenet', heads_only=False)
model = model_preprocess(model)
model.compile('nadam', 'categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
def train(lr, epochs, batch_size=10, augment=True):
    gen_train = aug_gen(X_train, y_train, batch_size=batch_size, augment=augment)
#     gen_val = aug_gen(X_test, y_test, batch_size=batch_size)
    keras.backend.set_value(model.optimizer.lr, lr)
    model.fit_generator(
        gen_train, steps_per_epoch=len(X_train)//batch_size,
        validation_data=(X_test, y_test),
        epochs=epochs, class_weight={0: 1, 1: 2})
train(1e-6, 1)
train(1e-4, 9)
# train(1e-5, 5)


In [ ]:
pred = model.predict((X_test.astype(np.float32)))
np.count_nonzero(pred.argmax(-1) == y_test.argmax(-1)) / y_test.shape[0]
print(confusion_matrix(y_test.argmax(-1), pred.argmax(-1)))

In [ ]:
# Test on val data
plt.figure(figsize=(6,6))
found = {i: 0 for i in "TP,FP,FN,TN".split(",")}
i = 0
while not all(found.values()):
    i += 1
    idx = np.random.choice(range(X_test.shape[0]))
    try:
        pred = model.predict(X_test[idx:idx+1].astype(np.float32))[0]
    except NameError:
        pred = None
    image, mask = X_test[idx], y_test[idx]
    image = np.clip(100*np.log10(1+image), 0, 255).astype(np.uint8)
    
    if pred is not None:
        pred_class = pred.argmax(-1)
        gt_class = mask.argmax(-1)
        TP = pred_class and pred_class == gt_class
        FP = pred_class and pred_class != gt_class
        TN = not pred_class and pred_class == gt_class
        FN = not pred_class and pred_class != gt_class
        if not found['TP'] and TP:
            plt.subplot(2, 2, 1)
            plt.imshow(image)
            plt.title("TP")
            found['TP'] = 1
        if not found['TN'] and TN:
            plt.subplot(2, 2, 4)
            plt.imshow(image)
            plt.title("TN")
            found['TN'] = 1
        if not found['FP'] and FP:
            plt.subplot(2, 2, 2)
            plt.imshow(image)
            plt.title("FP")
            found['FP'] = 1
        if not found['FN'] and FN:
            plt.subplot(2, 2, 3)
            plt.imshow(image)
            plt.title("FN")
            found['FN'] = 1
    if i == len(y_test):
        break
        

plt.tight_layout()


In [ ]:
# Test on test_blind
pred = None
while pred != 1:
    image = get_random_blind()
    image_show = np.clip(10*np.log10(1+image), 0, 1)
    pred = int(model.predict(image[np.newaxis]).argmax(-1)[0])

plt.figure(figsize=(4,4))
plt.imshow(image*3)
plt.title(str(pred));


In [ ]:
def save_model(model, dest_dir):
    model.save_weights(os.path.join(dest_dir, "weights.h5"))
    with open(os.path.join(dest_dir, "model.json"), "w") as file:
        file.write(model.to_json())
    
def load_model(src_dir):
    from keras.models import model_from_json
    with open(os.path.join(src_dir, "model.json"), "r") as file:
        model_json = file.read()
    model = model_from_json(model_json)
    model.load_weights(os.path.join(src_dir, "weights.h5"))
    return model
    
    

In [ ]:
save_model(model, dest_dir="/data/standard/ColorCheckerRECommended/model")
# model = load_model(src_dir="/data/standard/ColorCheckerRECommended/model")